In [4]:
import pandas as pd

In [5]:
dataset = pd.DataFrame(data={'Color': ['G','Y','R','R','Y'],
                             'Diameter': [3,3,1,1,3],
                             'Label': ['A','A','G','G','L']})

In [6]:
dataset

,Color,Diameter,Label
0,G,3,A
1,Y,3,A
2,R,1,G
3,R,1,G
4,Y,3,L


# Algorithm(steps) :-

Build_Tree()

    --> Find Best split

        - Find current Impurity

        - For each column-value combination:

            * Create a question object containing values for column and val. (To be used later for partitioning)

            * Partition data into left and right branches

            * If either partition size == 0, then skip current interation

            * information_gain = current_impurity - average_impurity of partitioned left and right branches

            * If information_gain > best_gain, then best_gain, best_question = information_gain, question

        - return best_gain, best_question

    --> If gain == 0 then return as Leaf_node(row)
    --> left, right = partition(rows)
    --> left branch = Build_Tree(left)
    --> right branch = Build_Tree(right)
    --> return Decision_Node(best_question, left branch, right branch)

In [ ]:
class Question:
    
    def __init__(self, col, value):
        self.column = col
        self.value = value

In [ ]:
def gini(rows):
    class_count = {}
    for row in rows:
        label = row[-1]
        if label not in class_count.items():
            class_count[label] = 0
        class_count[label] += 1
    impurity = 1
    for each in class_count.keys():
        impurity -= (class_count[each] / len(rows))**2
    return impurity

In [ ]:
def partition(rows, question):
    left_rows, right_rows = question.match()

In [ ]:
def best_split(rows):
    current_impurity = gini(rows)
    
    for col in range(n_features-1):
        values = set([row[col] for row in rows])
        
        for val in values:
            # Creating Question Class/object instead of function because (col and val) values 
            # need to be stored for each column and value combination, which will be used later
            question = Question(col, val)
            
            left_rows, right_rows = partition(rows, question)
            

In [ ]:
def build_tree(rows):
    gain, question = best_split(rows)